Here we will show an example of how to run deepblast on simulation data.

In [1]:
import os
from deepblast.sim import hmm_alignments
import argparse
import numpy as np

We will first simulate multiple sequences from a single PFam family.
The resulting generated alignments will be used to train the model.

In [2]:
hmm = '../data/zf-C2H2.hmm'
n_alignments = 100
np.random.seed(0)
align_df = hmm_alignments(n=40, seed=0, n_alignments=n_alignments, hmmfile=hmm)

cols = [
    'chain1_name', 'chain2_name', 'tmscore1', 'tmscore2', 'rmsd',
    'chain1', 'chain2', 'alignment'
]
align_df.columns = cols

The simulated alignments will be split into training / testing and validation.

In [3]:
parts = n_alignments // 10
train_df = align_df.iloc[:parts * 8]
test_df = align_df.iloc[parts * 8:parts * 9]
valid_df = align_df.iloc[parts * 9:]

# save the files to disk.
train_df.to_csv('data/train.txt', sep='\t', index=None, header=None)
test_df.to_csv('data/test.txt', sep='\t', index=None, header=None)
valid_df.to_csv('data/valid.txt', sep='\t', index=None, header=None)

We will prepare the environment to make sure that the appropriate output directories exist to store the results.

In [4]:
from deepblast.trainer import LightningAligner
from pytorch_lightning import Trainer

output_dir = 'simulation_results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [5]:
os.getcwd()

'/home/juermieboop/Documents/research/garfunkel/ipynb'

We will now create the arguments.  Below is the way to create this in a python environment.
This can also be recreated on a standard command line interface.

In [6]:
args = [
    '--train-pairs', f'{os.getcwd()}/data/train.txt',
    '--test-pairs', f'{os.getcwd()}/data/test.txt',
    '--valid-pairs', f'{os.getcwd()}/data/valid.txt',
    '--output-directory', output_dir,
    '--epochs', '15',
    '--batch-size', '10',   
    '--num-workers', '1',
    '--learning-rate', '1e-5',
    '--layers', '2',
    '--visualization-fraction', '1',
    '--loss', 'cross_entropy',
    '--scheduler', 'none',
    '--gpus', '1'
]
parser = argparse.ArgumentParser(add_help=False)
parser = LightningAligner.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)

We will then initialize the alignment model with the parameters we specified earlier.

In [7]:
model = LightningAligner(args)

We can now train the model.

In [8]:
trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=10,
    # profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)

trainer.fit(model)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | aligner | NeedlemanWunschAligner | 34 M  
/home/juermieboop/miniconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/juermieboop/miniconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of

1

The model diagnostics can be directly visualized in Tensorboard. Here, we show the losses, the accuracy and the alignment results.

In [9]:
!ls lightning_logs

version_0    version_16  version_33  version_50  version_68  version_85
version_1    version_17  version_34  version_51  version_69  version_86
version_10   version_18  version_35  version_52  version_7   version_87
version_100  version_19  version_36  version_53  version_70  version_88
version_101  version_2	 version_37  version_54  version_71  version_89
version_102  version_20  version_38  version_55  version_72  version_9
version_103  version_21  version_39  version_56  version_73  version_90
version_104  version_22  version_4   version_57  version_74  version_91
version_105  version_23  version_40  version_58  version_75  version_92
version_106  version_24  version_41  version_59  version_76  version_93
version_107  version_25  version_42  version_6	 version_77  version_94
version_108  version_26  version_43  version_60  version_78  version_95
version_109  version_27  version_44  version_61  version_79  version_96
version_11   version_28  version_45  version_62  version_8   versio

In [10]:
%load_ext tensorboard

In [11]:
%tensorboard --logdir lightning_logs

Reusing TensorBoard on port 6006 (pid 4355), started 9:57:10 ago. (Use '!kill 4355' to kill it.)

And we did this with just a few million parameters

In [12]:
model.aligner

NeedlemanWunschAligner(
  (lm): BiLM(
    (embed): Embedding(22, 21, padding_idx=21)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): ModuleList(
      (0): LSTM(21, 1024, batch_first=True)
      (1): LSTM(1024, 1024, batch_first=True)
    )
    (linear): Linear(in_features=1024, out_features=21, bias=True)
  )
  (match_embedding): StackedRNN(
    (embed): Embedding(21, 512, padding_idx=20)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): LSTM(512, 512, num_layers=2, batch_first=True, bidirectional=True)
    (proj): Linear(in_features=1024, out_features=512, bias=True)
  )
  (gap_embedding): StackedRNN(
    (embed): Embedding(21, 512, padding_idx=20)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): LSTM(512, 512, num_layers=2, batch_first=True, bidirectional=True)
    (proj): Linear(in_features=1024, out_features=512, bias=True)
  )
  (nw): NeedlemanWunschDecoder()
)

In [13]:
!ls lightning_logs/version_70/checkpoints

'epoch=59.ckpt'


In [14]:
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
checkpoint_dir = 'lightning_logs/version_70/checkpoints'
path = f'{checkpoint_dir}/epoch=59.ckpt'
model = LightningAligner.load_from_checkpoint(path)

In [15]:
from deepblast.dataset.dataset import states2matrix, tmstate_f
import numpy as np
import torch
from torch.nn.utils.rnn import pack_sequence
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
i = 9
test = align_df
x = str.encode(test.iloc[i]['chain1'])
y = str.encode(test.iloc[i]['chain2'])
states = test.iloc[i]['alignment']
x_ = torch.Tensor(model.tokenizer(x)).long().unsqueeze(0)
y_ = torch.Tensor(model.tokenizer(y)).long().unsqueeze(0)
x_ = pack_sequence(x_, enforce_sorted=False).cuda()
y_ = pack_sequence(y_, enforce_sorted=False).cuda()
states = list(map(tmstate_f, states))
A = states2matrix(states)

In [17]:
model = model.cuda()
aln, theta, gap = model.forward(x_, y_)

KeyError: 0

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(16, 3))
sns.heatmap(A, ax=ax[0])
sns.heatmap(aln.cpu().detach().numpy().squeeze(), ax=ax[1])
sns.heatmap(np.log(theta.cpu().detach().numpy().squeeze()), ax=ax[2])
sns.heatmap(gap.cpu().detach().numpy().squeeze(), ax=ax[3])

In [ ]:
from deepblast.dataset.alphabet import Uniprot21
from deepblast.dataset.dataset import decode
from deepblast.score import alignment_text, roc_edges

In [ ]:
from deepblast.dataset.dataset import states2edges

#pred_edges = list(zip(pred_y, pred_x))
pred_edges = states2edges(pred_states)
true_edges = states2edges(truth_states)
stats = roc_edges(true_edges, pred_edges)

In [ ]:
gen = model.aligner.traceback(x_, y_)
decoded, _ = next(gen)
pred_x, pred_y, pred_states = list(zip(*decoded))
pred_states = np.array(list(pred_states))
truth_states = np.array(list(states))                    
text = alignment_text(x.decode('utf-8'), y.decode('utf-8'), pred_states, truth_states, stats)

In [ ]:
print(text)

In [ ]:
px, py = zip(*pred_edges)
tx, ty = zip(*true_edges)

plt.plot(px, py, label='pred')
plt.plot(tx, ty, label='truth')

In [ ]:
from deepblast.dataset.dataset import state_diff_f
def states2edges(states):
    """ Converts state string to bipartite matching. """
    prev_s, next_s = states[:-1], states[1:]
    transitions = list(zip(prev_s, next_s))
    state_diffs = np.array(list(map(state_diff_f, transitions)))
    coords = np.cumsum(state_diffs, axis=0).tolist()
    coords = [(0, 0)] + list(map(tuple, coords))
    return coords
pe = states2edges(list(pred_states))
te = states2edges(list(truth_states))

In [ ]:
px, py = zip(*pe)
tx, ty = zip(*te)

plt.plot(px, py, label='pred')
plt.plot(tx, ty, label='truth')

In [ ]:
pe

In [ ]:
te

In [ ]:
truth_states

In [ ]:
pred_states